# Bonus Assignment 1

This is a Bonus Assignment try to complete all other Assignments before you reach here 

Try First Attempting Question 2 and then Question 1 


# Question 1
Solve this question on Kaggle! 
Download the notebook and submit it separately along with the other questions.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Use everything that you have learnt so far.

Analyse the data. Drop, scale or normalize features. Merge features if they correlated to make some meaningful features. 

Try out all the regression models. Use cross validation to find best parameters for each model.

In [ ]:
# SUBMITTED SEPERATELY

# Question 2 : Regression using SVM and Tree Algorithms

In this question, we will be using the **insurance.csv** file which contain information on insurance charges based on the following informations: age,sex,bmi,region,number of children and whether the person is a smoker or not. You need to predict the charges based on the information given.

### 1. Load the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./insurance.csv')

In [2]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### 2. Separate the numerical and categorical columns.
### 3. Label Encode the categorical columns.
### 4. Scale the numerical columns. (Scale the charges separately so that you can calculate errors afterwards.)

In [9]:
numericalCols = ['age', 'bmi', 'children']
categoricalCols = ['sex', 'smoker', 'region']

In [48]:
df_categorical = df[categoricalCols]
df_numerical = df[numericalCols]

In [54]:
map_sex = {'male':0, 'female':1}
map_smoker = {'no':0, 'yes':1}
map_region = {'southeast':0, 'southwest':1, 'northwest':2, 'northeast':3}

df_encoded = pd.DataFrame(columns=categoricalCols)

df_encoded['sex'] = (df_categorical['sex'].map(map_sex)).astype(int)
df_encoded['smoker'] = (df_categorical['smoker'].map(map_smoker)).astype(int)
df_encoded['region'] = (df_categorical['region'].map(map_region)).astype(int)

In [55]:
df_encoded.head().dtypes

sex       int32
smoker    int32
region    int32
dtype: object

In [61]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_numerical)

df_scaled = pd.DataFrame(scaled_data, columns = df_numerical.columns)

In [63]:
df_modified = pd.concat([df_scaled, df_encoded], axis=1)
df_modified.head()

,age,bmi,children,sex,smoker,region
0,-1.438764,-0.453320,-0.908614,1,1,1
1,-1.509965,0.509621,-0.078767,0,0,0
2,-0.797954,0.383307,1.580926,0,0,0
3,-0.441948,-1.305531,-0.908614,0,0,2
4,-0.513149,-0.292556,-0.908614,0,0,2


In [64]:
X = np.array(df_modified)
y = df['charges']

### 5. Split the data.

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 6. Support Vector Regressor

Here , you will use the SVR model from sklearn.svm and fit it on the training data. Then predict on the test data and calaculate MAE, MSE. But...

The SVR class contains many hyperparameters, example : kernel can have the following values : linear, rbf, poly, sigmoid.

Use **RandomizedSearchCV** from sklearn.model_selection , create a dictionary with keys 'kernel' and 'gamma' . As values of the keys, create a list of some possible values. Run a 3-fold cross validation test (cv=3) and find the best parameters. Then initiate the SVR model with those parameters.

In [73]:
from sklearn.svm import SVR

model = SVR(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [74]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(f"MAE = {mean_absolute_error(y_true=y_test, y_pred=y_pred)}")
print(f"MSE = {mean_squared_error(y_true=y_test, y_pred=y_pred)}")
print(f"R2 = {r2_score(y_true=y_test, y_pred=y_pred)}")

MAE = 8295.675066922398
MSE = 170714648.24342456
R2 = -0.1015862933986782


In [86]:
param_distributions = {
    'gamma': np.arange(1, 10),
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

In [87]:
from sklearn.model_selection import RandomizedSearchCV

randomized_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=3,  
    random_state=42
)
randomized_search.fit(X_train, y_train)

c:\Users\Tejas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 36 is smaller than n_iter=50. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=SVR(kernel='linear'), n_iter=50,
                   param_distributions={'gamma': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'kernel': ['linear', 'rbf', 'poly',
                                                   'sigmoid']},
                   random_state=42, scoring='neg_mean_absolute_error')

In [85]:
best_model = randomized_search.best_estimator_

y_pred = best_model.predict(X_test)

accuracy = r2_score(y_test, y_pred)
print("Best Hyperparameters:", randomized_search.best_params_)
print("Test Set Accuracy:", accuracy)

Best Hyperparameters: {'kernel': 'poly', 'gamma': 10.0}
Test Set Accuracy: 0.8337759130828561
